In [1]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2

In [2]:
# def upsertHistoricalDailyData(exchange:str, symbol:str, start_date:str, end_date:str)-> None:
#     ticker = ''
#     if exchange == 'NSE':
#         ticker = symbol + ".NS"
#     print(ticker)
    

In [3]:
# ticker = "^NSEI"  #for nifty 50
ticker = "RELIANCE.NS"
start_date = "2024-12-05"
end_date = "2024-12-31"

# Download the historical data for Nifty Index
data = yf.download(ticker, start=start_date, end=end_date)
data


[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,,
2024-12-05,1322.050049,1322.050049,1329.949951,1306.150024,1314.349976,16858723
2024-12-06,1311.550049,1311.550049,1323.900024,1310.000000,1323.900024,9037514
2024-12-09,1295.150024,1295.150024,1315.000000,1293.099976,1303.000000,14650002
2024-12-10,1284.849976,1284.849976,1294.900024,1280.199951,1289.250000,16043729
2024-12-11,1278.199951,1278.199951,1290.000000,1276.050049,1285.000000,11724373
2024-12-12,1262.900024,1262.900024,1278.199951,1260.599976,1270.000000,20906813
2024-12-13,1272.849976,1272.849976,1275.199951,1239.599976,1260.000000,28630222
2024-12-16,1268.300049,1268.300049,1281.000000,1266.550049,1275.000000,9486781


In [4]:
# POSTGRES_HOST=localhost
# POSTGRES_USERNAME=postgres
# POSTGRES_PASSWORD=password
# POSTGRES_PORT= 5432
# POSTGRES_DATABASE=mydatabase

conn = psycopg2.connect(
    database='mydatabase', user='postgres', password='password', host='localhost', port= '5432'
)

cursor = conn.cursor()
stockkey = 'NSE:RELIANCE'
print(f"helllo {stockkey}")
for row in data.itertuples():
    # any transformation can be applied here

    #ts.strftime('%Y-%m-%d')
    priceDate = row.Index.strftime('%Y-%m-%d')
    adjustedClosingPrice = row._1
    closingPrice = row._2
    highPrice = row._3
    lowPrice = row._4
    openPrice = row._5
    volume = row._6
    sql = f"insert into WealthManager.DailyPriceMaster (stockkey,  adjcloseprice, closeprice, openprice, high, low, volume, priceDate) values ('{stockkey}', {adjustedClosingPrice} , {closingPrice}, {openPrice}, {highPrice}, {lowPrice},{volume}, '{priceDate}') on conflict(stockkey, priceDate) do nothing "
    cursor.execute(sql)

conn.commit()
    

    

helllo NSE:RELIANCE


https://www.pauldesalvo.com/how-to-flatten-multiindex-columns-into-a-single-index-dataframe-in-pandas/

https://www.geeksforgeeks.org/reverting-from-multiindex-to-single-index-dataframe-in-pandas/

In [10]:
# import yfinance as yf
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# ticker = "^NSEI"
# start_date = "2009-12-31"
# end_date = "2023-04-01"

# # Download the historical data for Nifty Index
# data = yf.download(ticker, start=start_date, end=end_date)

# # Resample the data on a monthly basis
# data_monthly = data.resample('M').last()

# # Calculate the monthly returns
# monthly_returns = data_monthly['Adj Close'].pct_change()

# # Convert monthly returns to a pandas DataFrame
# monthly_returns_df = pd.DataFrame(monthly_returns)

# # Pivot the DataFrame to create a matrix of monthly returns by year and month
# monthly_returns_matrix = monthly_returns_df.pivot_table(values='Adj Close', index=monthly_returns_df.index.year, columns=monthly_returns_df.index.month)

# # Set the column names to the month names
# monthly_returns_matrix.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# # Calculate the sum of monthly returns for each year
# yearly_returns = monthly_returns_df.groupby(monthly_returns_df.index.year)['Adj Close'].sum()

# # Add the yearly returns to the matrix as a new column
# monthly_returns_matrix['Yearly'] = yearly_returns

# # Set the font scale
# sns.set(font_scale=1.2)

# # Plot the heatmap using seaborn
# plt.figure(figsize=(14, 12))
# sns.heatmap(monthly_returns_matrix, annot=True, cmap='RdYlGn', center=0, fmt='.2%', cbar=False)
# plt.title('Nifty Monthly and Yearly Returns by Year and Month', fontsize=20)
# plt.xlabel('Month', fontsize=14)
# plt.ylabel('Year', fontsize=14)
# plt.show()